This note book aim to get data about house price from http://house.ksou.cn/




In [ ]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver= webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options


# process

In [ ]:
folder_path =  "drive/MyDrive/SIT764/Property/data/"

In [ ]:
# Get suburb information at Geelong
suburb_data = pd.read_csv(folder_path + "Suburb_Profile.csv")

In [ ]:
suburb_list = list(suburb_data["Suburb"].values)
suburb_list[4] = 'BreakWater'
suburb_list[24] = "RippleSide"
suburb_code = list(suburb_data["Suburb_Code"].values)
suburb_dic = {suburb_list[i]: suburb_code[i] for i in range(len(suburb_list))}

In [ ]:
headers = ["Address", "Sell Price", "Sell Date", "Last Sell Price", "Last Sell Date", "Auction","Rent Amount PW", "Rent Date", "Land Size SQM", "Building Size SQM", "Bedrooms", "Toilets", "Car", "Suburb", "Suburb_Code"]

In [ ]:
new_dataset = pd.DataFrame([], columns = headers)

In [ ]:
# Get data from house.speakingsame.com
def generate_data(suburb_name):
  url = f"http://house.speakingsame.com/p.php?q={suburb_name.replace(' ', '+')}%2C+VIC"
  driver.get(url)
  sleep(5)
  num_page = 0
  data_set = []
  while num_page < 100:
    try:
      for i in range(2,12):
        auction = np.nan
        browser = driver.find_element_by_xpath(f"/html/body/table[3]/tbody/tr/td[2]/div/table[{i}]/tbody/tr/td[2]/table/tbody")
        lines = browser.text.splitlines()
        address = lines[1]
        current_sale = lines[2].split()
        if len(current_sale) > 5:
          if "Auction" in lines[2]:
            auction = 1
          current_date = current_sale[3] + " " + current_sale[4] + " "+ current_sale[5][:4]
        else:
          if "Auction" in lines[2]:
            auction = 1
          current_date = current_sale[3] + " "+ current_sale[4][:4]
          
        last_sale, last_date = np.nan, np.nan
        rent = np.nan
        rent_date = np.nan
        bbc = [np.nan, np.nan, np.nan]
        land_size = np.nan
        building_size = np.nan
        
        for j in range(3, len(lines)):
          if "Last Sold" in lines[j]:
            last_sale_info = lines[j].split()

            if len(last_sale_info) == 5:
              if "Auction" in lines[j]:
                auction = 1
              last_sale, last_date = last_sale_info[2][1:], last_sale_info[4][:4]
              
            elif len(last_sale_info) == 6:
              if "Auction" in lines[j]:
                auction = 1
              last_sale, last_date = last_sale_info[2][1:], last_sale_info[4] + " " + last_sale_info[5][:4]

          elif "Rent" in lines[j]:
            rent_info = lines[j].split()
            rent, rent_date = rent_info[1][:-2][1:], rent_info[3] + " " + rent_info[4]

          elif "House:" in lines[j]:
            house = lines[j].split()
            for i in range(len(house)-1):
              bbc[i] = house[i+1]

          elif "Land size" in lines[j] or "Building size" in lines[j]:
            sizes = lines[j].split(' | ')
            for size in sizes:
              if "Land size" in size:
                land_size = size.split()[2]
              else:
                building_size = size.split()[2]
        output = [address, current_sale[1][1:], current_date, last_sale, last_date, auction, rent, rent_date, land_size, building_size, bbc[0], bbc[1], bbc[2], suburb_name,suburb_dic[suburb_name]]
        data_set.append(output)
      if num_page == 0:
        driver.get(f"http://house.speakingsame.com/p.php?q={suburb_name.replace(' ', '+')}&p=1&s=1&st=&type=&count=300&region={suburb_name.replace(' ', '+')}&lat=0&lng=0&sta=vic")
        num_page += 1
      else:
        driver.get(f"http://house.speakingsame.com/p.php?q={suburb_name.replace(' ', '+')}&p={num_page+1}&s=1&st=&type=&count=300&region={suburb_name.replace(' ', '+')}&lat=0&lng=0&sta=vic")
        num_page += 1
    except:
      break
  driver.close()
  print(f"{suburb_name}:{num_page}")
  data_set = pd.DataFrame(data_set, columns=headers)
  return data_set

# Get data 




In [ ]:
Store the dataset of each suburb
for subname in suburb_list:
  data = generate_data(f'{subname}')
  data.to_csv(folder_path + f"subdata/{subname}.csv",index=False) 